**Aploading Data to S3**

In [ ]:
# Upload data to S3
# Upload the CV_Data to cv_data folder: s3://reactorlab/Amanda/cv_data/
# Upload the Wavetronix Data to wavetronix_filtered folder: s3://reactorlab/Amanda/wavetronix_filtered/
# Upload locations data to wavetronix_loations foler: s3://reactorlab/Amanda/wavetronix_locations/

**Creating Tables in Athena**

In [ ]:
#  CV_Data Table
CREATE EXTERNAL TABLE IF NOT EXISTS cv_data (
  datapointid STRING,
  journeyed STRING,
  capturedtimestamp TIMESTAMP,
  latitude DOUBLE,
  longitude DOUBLE,
  route_id STRING,
  segment_start_measure DOUBLE,
  hour INT
)
STORED AS PARQUET
LOCATION 's3://reactorlab/Amanda/cv_data/';


In [ ]:
# Create the Wavetronix table for locations
CREATE EXTERNAL TABLE IF NOT EXISTS wavetronix (
  device_id STRING,
  routeid STRING,
  latitude DOUBLE,
  longitude DOUBLE
)
STORED AS PARQUET
LOCATION 's3://reactorlab/Amanda/wavetronix_locations/';


In [ ]:
# Create the Wavetronix table for filtered data
CREATE EXTERNAL TABLE IF NOT EXISTS wavetronix_data (
  device_id STRING,
  lane_id STRING,
  lane_count INT,
  link_direction STRING,
  cst_time TIMESTAMP
)
STORED AS PARQUET
LOCATION 's3://reactorlab/Amanda/wavetronix_filtered/';

**Calculating Penetration Rates**

In [ ]:
# Consolidated Athena Query for CV and Wavetronix Data Processing with Percentage Calculation
WITH cv_with_sensor AS (
    SELECT
        cv.datapointid,
        cv.journeyid,
        cv.capturedtimestamp,
        cv.latitude AS cv_latitude,
        cv.longitude AS cv_longitude,
        cv.route_id,
        cv.hour,
        wl.device_id AS sensor_device_id,
        wl.latitude AS sensor_latitude,
        wl.longitude AS sensor_longitude,
        -- Calculate approximate distance for proximity matching
        sqrt(power(cv.latitude - wl.latitude, 2) + power(cv.longitude - wl.longitude, 2)) AS distance
    FROM
        raghu_cv_data AS cv
    JOIN
        raghu_wavetronix_locations AS wl
    ON
        cv.route_id = wl.route_id
),
cv_hourly_count AS (
    SELECT
        sensor_device_id AS device_id,
        hour,
        COUNT(DISTINCT journeyid) AS cv_count
    FROM
        cv_with_sensor
    WHERE distance < 0.01 -- Filter by distance threshold for nearest sensor
    GROUP BY
        sensor_device_id, hour
),
wavetronix_hourly_count AS (
    SELECT
        device_id,
        hour(cst_time) AS hour,
        SUM(lane_count) AS total_vehicle_count,
        MAX(lane_direction) AS link_direction -- Assuming one direction per device per hour
    FROM
        raghu_wavetronix_data
    GROUP BY
        device_id, hour(cst_time)
)
SELECT
    cv.device_id,
    cv.hour,
    cv.cv_count,
    wt.total_vehicle_count,
    wl.latitude AS sensor_latitude,
    wl.longitude AS sensor_longitude,
    wt.link_direction,
    -- Calculate proportion of CVs
    IF(wt.total_vehicle_count > 0, CAST(cv.cv_count AS DECIMAL(10, 4)) / CAST(wt.total_vehicle_count AS DECIMAL(10, 4)), 0) AS cv_proportion,
    -- Calculate percentage of CV proportion
    IF(wt.total_vehicle_count > 0, (CAST(cv.cv_count AS DECIMAL(10, 3)) / CAST(wt.total_vehicle_count AS DECIMAL(10, 3))) * 100, 0) AS cv_percentage
FROM
    cv_hourly_count AS cv
JOIN
    wavetronix_hourly_count AS wt
ON
    cv.device_id = wt.device_id
    AND cv.hour = wt.hour
JOIN
    raghu_wavetronix_locations AS wl
ON
    cv.device_id = wl.device_id
ORDER BY
    cv.device_id, cv.hour;